# Training on AI Platform with a custom container

In [1]:
import os
import time

In [2]:
TRAINING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/training.csv'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = 'gs://jk-demo-artifacts'
JOBDIR_BUCKET = 'gs://jk-demo-jobdir'

## Create a training container image
### Configure Cloud Build to use Kaniko

In [3]:
!gcloud config set builds/use_kaniko True

Updated property [builds/use_kaniko].


### Create a Dockerfile

In [4]:
TRAINING_IMAGE_FOLDER = '../training_image'

os.makedirs(TRAINING_IMAGE_FOLDER, exist_ok=True)

In [5]:
%%writefile $TRAINING_IMAGE_FOLDER/Dockerfile

FROM gcr.io/jk-demo1/sklearn-cpu:latest
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing ../training_image/Dockerfile


### Create a training script

In [7]:
%%writefile $TRAINING_IMAGE_FOLDER/train.py

import logging
import os
import subprocess
import sys
import joblib
import fire
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  # Configure a training pipeline
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  # Configure a parameter grid
  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  # Tune hyperparameters
  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../training_image/train.py


### Build the image

In [8]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
IMAGE_REPO_NAME="octane-regression-training"
IMAGE_TAG="latest"
IMAGE_URI="gcr.io/{}/{}:{}".format(PROJECT_ID, IMAGE_REPO_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $TRAINING_IMAGE_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 2.0 KiB before compression.
Uploading tarball of [../training_image] to [gs://jk-demo1_cloudbuild/source/1566349680.07-08ee599b76224927ab890cd57ab99e64.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-demo1/builds/6dccbb74-4fb0-46b7-8ba1-652ac06c3c7a].
Logs are available at [https://console.cloud.google.com/gcr/builds/6dccbb74-4fb0-46b7-8ba1-652ac06c3c7a?project=826865698127].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6dccbb74-4fb0-46b7-8ba1-652ac06c3c7a"

FETCHSOURCE
Fetching storage object: gs://jk-demo1_cloudbuild/source/1566349680.07-08ee599b76224927ab890cd57ab99e64.tgz#1566349680565089
Copying gs://jk-demo1_cloudbuild/source/1566349680.07-08ee599b76224927ab890cd57ab99e64.tgz#1566349680565089...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      
B

### Submit a training job

In [10]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"

N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

In [11]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--master-image-uri $IMAGE_URI \
--scale-tier $SCALE_TIER \
-- \
--data_path $TRAINING_DATA_PATH  \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190821_011018] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190821_011018

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190821_011018
jobId: JOB_20190821_011018
state: QUEUED


In [12]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-21T01:10:21Z'
etag: 0Pq62cezM2w=
jobId: JOB_20190821_011018
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190821_011018
  masterConfig:
    imageUri: gcr.io/jk-demo1/octane-regression-training:latest
  region: us-central1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190821_011018?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190821_011018&project=jk-demo1
INFO	2019-08-21 01:10:21 +0000	service		Validating job requirements...
INFO	2019-08-21 01:10:21 +0000	service		Job creation request has been successfully validated.
INFO	2019-08-21 01:10:21 +0000	service		Job JOB_20190821_011018 is queued.
INFO	2019-08-21 01:10:22 +0000	service		Waiting for job to be provisioned.

### Copy the trained model to model repo

In [14]:
MODEL_NAME = "octane-regressor-container"

!gsutil cp $JOBDIR_BUCKET/$JOB_NAME/model.joblib $ARTIFACT_BUCKET/models/$MODEL_NAME/model.joblib

Copying gs://jk-demo-jobdir/JOB_20190821_011018/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     


## Download and test the model

In [15]:
LOCAL_PATH = '/tmp/model.joblib'

!gsutil cp $ARTIFACT_BUCKET/models/$MODEL_NAME/model.joblib $LOCAL_PATH 

Copying gs://jk-demo-artifacts/models/octane-regressor-container/model.joblib...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     


In [16]:
import joblib
import pandas as pd

df_test = pd.read_csv(TESTING_DATA_PATH)
predictor = joblib.load(LOCAL_PATH)

y = df_test.octane
X = df_test.drop('octane', axis=1)

y_hat = predictor.predict(X)
print(list(zip(y, y_hat)))

[(88.65, 88.59553605661218), (88.7, 88.84093111257747), (87.3, 87.35784874687317), (88.85, 88.73399177287303), (85.1, 85.26014664670035), (88.4, 88.12292248697285)]


/home/jupyter/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator PCA from version 0.21.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Ridge from version 0.21.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter/.local/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at 